In [76]:
import torch
from torch import nn
from torch.nn import functional as F
from torch import optim
import dlc_practical_prologue as prologue

mini_batch_size = 100
N = 1000
train_input, train_target, train_classes, test_input, test_target, test_classes = prologue.generate_pair_sets(N)

In [77]:
#normalize the input
train_input/=255
test_input/=255

In [78]:
print_shapes_Net = False

In [79]:
class Net_auxiliary_loss(nn.Module):
    def __init__(self):
        super(Net_auxiliary_loss, self).__init__()
        self.conv1 = nn.Conv2d(2, 32, kernel_size=3)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3)
        self.fc1 = nn.Linear(256, 10)
        self.fc2 = nn.Linear(10, 2)

    def forward(self, x):
        if print_shapes_Net:
            print("initial", x.shape) #100 2 14 14
        x = F.relu(F.max_pool2d(self.conv1(x), kernel_size=2, stride=2))
        if print_shapes_Net:
            print("conv1",x.shape) #100 32 6 6
        x = F.relu(F.max_pool2d(self.conv2(x), kernel_size=2, stride=2))
        if print_shapes_Net:
            print("conv2",x.shape) #100 64 2 2
        x = F.relu(self.fc1(x.view(-1, 256)))
        x_class = x #100 10
        if print_shapes_Net:
            print("fc1",x.shape) #100 10
        x = self.fc2(x)
        if print_shapes_Net:
            print("final",x.shape) # 100 2 
        return x_class, x

In [80]:
def train_model(model, train_input, train_classes, train_target, mini_batch_size, lr, nb_epoch):
    criterion = nn.MSELoss() #CHANGE LOSS TO CROSS ENTROPY?
    optimizer = optim.Adam(model.parameters(), lr)
    
    for e in range(nb_epoch):
        sum_loss = 0
        for b in range(0, train_input.size(0), mini_batch_size):
          
            output_class, output_target = model(train_input.narrow(0, b, mini_batch_size))
            
            loss_class = criterion(output_class, train_classes.narrow(0, b, mini_batch_size))
            loss_target = criterion(output_target, train_target.narrow(0, b, mini_batch_size))
            model.zero_grad()
            loss = loss_class*0.5 + loss_target
            loss.backward()
            optimizer.step()
            
            sum_loss = sum_loss + loss.item()
        print(e, sum_loss)

In [81]:
def compute_nb_errors_targets(model, input, target):
    nb_errors = 0
    _, output = model(input)
    _, predicted_target = output.max(1) #max probabilities of target
    
    for b in range(1000):
        if target[b,int(predicted_target[b])] <= 0:
            nb_errors = nb_errors + 1
            
    return nb_errors

In [82]:
def compute_nb_errors_classes(model, input, target):
    nb_errors = 0

    output,_ = model(input)
    _, predicted_classes = output.max(1)
    print(output[0])
    print(target[0])

    for b in range(input.shape[0]):
        if target[b, predicted_classes[b]] <= 0:
            nb_errors = nb_errors + 1

    return nb_errors

In [83]:
#train_target[1000,1]
new_train_target = torch.empty(1000,2)
new_test_target = torch.empty(1000,2)
for i in range(1000):
    if train_target[i] == 1 :
        new_train_target[i,0] = 0
        new_train_target[i,1] = 1
        
    else:
        new_train_target[i,0] = 1
        new_train_target[i,1] = 0
        
    if test_target[i] == 1:
        new_test_target[i,0] = 0
        new_test_target[i,1] = 1
        
    else:
        new_test_target[i,0] = 1
        new_test_target[i,1] = 0

In [84]:
#train_classes[1000, 2]
new_train_classes = torch.zeros(1000, 10)
new_test_classes = torch.zeros(1000, 10)

for i in range(train_classes.shape[0]):
    new_train_classes[i][train_classes[i][0]] = 1
    new_train_classes[i][train_classes[i][1]] = 1

for i in range(test_classes.shape[0]):
    new_test_classes[i][test_classes[i][0]] = 1
    new_test_classes[i][test_classes[i][1]] = 1



In [85]:
####predict class of each digit
for k in range(1):
    model = Net_auxiliary_loss()
    lr = 0.005
    nb_epoch = 25
    train_model(model, train_input, new_train_classes, new_train_target, mini_batch_size, lr, nb_epoch)
    
    nb_train_errors = compute_nb_errors_targets(model, train_input, new_train_target)
    print('train error Net_auxiliary_loss {:0.2f}% {:d}/{:d}'.format((100 * nb_train_errors) / train_input.size(0),
                                                      nb_train_errors, train_input.size(0)))
    nb_test_errors = compute_nb_errors_targets(model, test_input, new_test_target)
    print('test error Net_auxiliary_loss {:0.2f}% {:d}/{:d}'.format((100 * nb_test_errors) / test_input.size(0),
                                                    nb_test_errors, test_input.size(0)))
    

0 4.882763028144836
1 3.9175607562065125
2 3.617906540632248
3 3.4353533685207367
4 3.219293534755707
5 2.994525760412216
6 2.8179134130477905
7 2.644164279103279
8 2.467716619372368
9 2.302626132965088
10 2.1410613656044006
11 2.008056089282036
12 1.8517899215221405
13 1.7347681522369385
14 1.580661579966545
15 1.4740663170814514
16 1.40530526638031
17 1.362161323428154
18 1.2960942015051842
19 1.248879760503769
20 1.2121433466672897
21 1.15548375248909
22 1.1570978984236717
23 1.187822625041008
24 1.2050592750310898
train error Net_auxiliary_loss 0.30% 3/1000
test error Net_auxiliary_loss 21.50% 215/1000


In [86]:
nb_train_errors_class = compute_nb_errors_classes(model, train_input, train_classes)
print('train error Net_auxiliary_loss {:0.2f}% {:d}/{:d}'.format((100 * nb_train_errors_class) / train_input.size(0),
                                                  nb_train_errors_class, train_input.size(0)))
nb_test_errors_class = compute_nb_errors_classes(model, test_input, test_classes)
print('test error Net_auxiliary_loss {:0.2f}% {:d}/{:d}'.format((100 * nb_test_errors_class) / test_input.size(0),
                                                nb_test_errors_class, test_input.size(0)))

tensor([0.0000, 0.0000, 0.0000, 0.0000, 0.0040, 0.0000, 0.8168, 0.0000, 0.2784,
        0.6731], grad_fn=<SelectBackward>)
tensor([5, 6])


IndexError: index 6 is out of bounds for dimension 1 with size 2